## Managing the transaction lifecycle on LUSID

In this notebook we show how you can use properties to manage the transaction lifecycle. For the purposes of this demonstration, we're using the [transaction](https://support.finbourne.com/what-is-a-transaction) entity but the same principles could be applied to LUSID's orders and allocations, or indeed any data entity which has properties. 

### Setup LUSID

In [1]:
# Import LUSID
import lusid.models as models
from lusidjam import RefreshingToken

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import pandas as pd
import numpy as np
import json
import requests
import os
import lusid
import lusidtools.cocoon.cocoon as cocoon
from lusidtools.cocoon.utilities import create_scope_id
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
import uuid

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

print("LUSID Environment Initialised")
print(
    "LUSID API Version: ",
    api_factory.build(lusid.api.ApplicationMetadataApi)
    .get_lusid_versions()
    .build_version,
)

LUSID Environment Initialised
LUSID API Version:  0.5.4275.0


In [2]:
# define a unqique scope

scope = create_scope_id()
portfolio_code = "EQUITY_UK"

# Load a mapping file for formatting DataFrame columns

with open(r"config/transaction_status.json") as mappings_file:
    transaction_status_mapping = json.load(mappings_file)

### 1) Introduction to transaction statuses

In the code below, we're going to assign some statuses to a transaction. For the purposes of this demo, we will use 4 statuses listed below. As with all LUSID properties, you are free to assign your own codes and values.

| Transaction Status  | Status description |
| :------------- | :------------- |
| Executed  | The transaction has been executed on the market|
| ComplianceReview  | The transaction might go to the Compliance team to review  |
| Confirmed  | Transaction details have been confirmed with the counterparty  |
| Settled  | Transaction has settled at the custodian/clearer  |

<br>

### 2) Create a new data type to hold acceptable values for the status

In this section, we create a new LUSID data type object to hold the list of acceptable values: Executed, ComplianceReview, Confirmed, and Settled. This data type will be used when creating the <b>transaction_status</b> property definition.

In [3]:
try:

    create_request = lusid.models.CreateDataTypeRequest(
        scope=scope,
        code="TxnStatusCodes",
        type_value_range="Closed",
        display_name="Available transaction status codes",
        description="List of allowable transaction status codes",
        value_type="String",
        acceptable_values=["Executed", "ComplianceReview", "Confirmed", "Settled"],
    )

    response = api_factory.build(lusid.api.DataTypesApi).create_data_type(
        request=create_request
    )

    print("Data Type Created")

except:

    response = api_factory.build(lusid.api.DataTypesApi).get_data_type(
        scope=scope, code="TxnStatusCodes"
    )

    print(response)

Data Type Created


### 3) Create a new property definition of transaction_status

Next we define a new <b>transaction_status</b> property which will be used on the transaction to set the allowed statuses. As you can see, we have passed the <b>data_type_id</b> which we created above.

In [4]:
try:
    api_factory.build(lusid.api.PropertyDefinitionsApi).create_property_definition(
        lusid.models.CreatePropertyDefinitionRequest(
            domain="Transaction",
            scope=scope,
            code="transaction_status",
            value_required=None,
            display_name="Transaction Status",
            data_type_id=lusid.ResourceId(scope=scope, code="TxnStatusCodes"),
            life_time=None,
        )
    )
except:
    pass

### 4) Create new portfolio with transactions

In the code below, we create a new portfolio called <b>EQUITY_UK</b> with two transactions from the <i>equity_transactions.csv</i> file. There is one equity trade and one subsciption of cash. The transactions have the <b>transaction_status</b> of <i>Executed</i> and <i>Confirmed</i> which are allowed values per the data type above.

In [5]:
transactions_file = r"data/equity_transactions.csv"
transactions_df = pd.read_csv(transactions_file)
transactions_df["portfolio_code"] = portfolio_code
transactions_df

,portfolio_code,portfolio_name,portfolio_base_currency,ticker,sedol,instrument_type,instrument_id,name,txn_id,txn_type,txn_trade_date,txn_settle_date,txn_units,txn_price,txn_consideration,currency,transaction_status,cash_transactions
0,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0002162385,SEDOL1,equity,EQ_1234,Aviva,trd_0001,Buy,02/01/2020,04/01/2020,120000,5,600000,GBP,Executed,NaN
1,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GBP,GBP,cash,GBP,GBP Cash,trd_0021,FundsIn,02/01/2020,04/01/2020,12000000,1,12000000,GBP,Confirmed,GBP


In [6]:
# The seed_data() function takes a file of transaction data
# and loads portfolios, instruments, and transactions into LUSID
# We use this function as a quick way of generating a demo portfolio

seed_data(
    api_factory,
    ["portfolios", "instruments", "transactions"],
    scope,
    transactions_file,
    "csv",
    mappings=transaction_status_mapping,
)

print(f"Portfolio {portfolio_code} has been created with transactions.")

Portfolio EQUITY_UK has been created with transactions.


### 5) Check transactions were loaded successfully

Check that the transactions loaded correctly by calling the [get transactions](https://www.lusid.com/docs/api/#operation/GetTransactions) method on the TransactionPortfolios API.

In [7]:
# Define the transactions portfolio API
transaction_portfolio_api = api_factory.build(lusid.api.TransactionPortfoliosApi)

# Call the get transactions method 
get_transaction_response = transaction_portfolio_api.get_transactions(
    scope=scope, code=portfolio_code)

# Scrape the response into a DataFrame
get_transactions_df = lusid_response_to_data_frame(
    get_transaction_response, rename_properties=True)

# Print the DataFrame
get_transactions_df

,transaction_id,type,instrument_identifiers.Instrument/default/ClientInternal,instrument_uid,transaction_date,settlement_date,units,transaction_price.price,transaction_price.type,total_consideration.amount,total_consideration.currency,exchange_rate,transaction_currency,SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),transaction_status(385c-a208-deb2-78-Properties),instrument_identifiers.Instrument/default/Currency
0,trd_0001,Buy,EQ_1234,LUID_ATFGUBHS,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,120000.0,5.0,Price,600000.0,GBP,1.0,GBP,EQUITY_UK,385c-a208-deb2-78,Executed,NaN
1,trd_0021,FundsIn,NaN,CCY_GBP,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,12000000.0,1.0,Price,12000000.0,GBP,1.0,GBP,EQUITY_UK,385c-a208-deb2-78,Confirmed,GBP


### 6) Correct the transaction status to <i>Matched</i> - this is an invalid value 

To show the data validation on <b>transaction_status</b> field, we now want to add an unacceptable value. For the purposes of this demo, we will correct the <b>transaction_status</b> to <i>Matched</i> which is not one of the 4 acceptable values.

In [8]:
transactions_df.at[0, "transaction_status"] = "Matched"
transactions_df

,portfolio_code,portfolio_name,portfolio_base_currency,ticker,sedol,instrument_type,instrument_id,name,txn_id,txn_type,txn_trade_date,txn_settle_date,txn_units,txn_price,txn_consideration,currency,transaction_status,cash_transactions
0,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0002162385,SEDOL1,equity,EQ_1234,Aviva,trd_0001,Buy,02/01/2020,04/01/2020,120000,5,600000,GBP,Matched,NaN
1,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GBP,GBP,cash,GBP,GBP Cash,trd_0021,FundsIn,02/01/2020,04/01/2020,12000000,1,12000000,GBP,Confirmed,GBP


As you can see, the request fails with an <b>InvalidTransactions</b> error: 

In [9]:
result = cocoon.load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=transactions_df,
    mapping_required=transaction_status_mapping["transactions"]["required"],
    mapping_optional=transaction_status_mapping["transactions"]["optional"],
    file_type="transactions",
    identifier_mapping=transaction_status_mapping["transactions"]["identifier_mapping"],
    property_columns=transaction_status_mapping["transactions"]["properties"],
    properties_scope=scope,
)

error_response = result["transactions"]["errors"][0].body
print("Error type: ", json.loads(error_response)["name"],"\n")
print("Transaction ID which created error: ", json.loads(error_response)["errorDetails"][0]["id"],"\n")
print("Error details: ", json.loads(error_response)["errorDetails"][0]["detail"],"\n")

Error type:  InvalidTransactions 

Transaction ID which created error:  trd_0001 

Error details:  Invalid value 'Matched' supplied to the field of data type 385c-a208-deb2-78/TxnStatusCodes. Validation error: The value is not in the list of acceptable values 

